<a href="https://colab.research.google.com/github/Srilakshmi8-git/Calorie_Prediction_System1/blob/main/calorie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import os
import cv2
import torch
import numpy as np
from segment_anything import sam_model_registry, SamPredictor
from tqdm import tqdm
import matplotlib.pyplot as plt

# Paths
IMAGE_ROOT = "/content/drive/MyDrive/Calorie_prediction_system/dataset/images"
MASK_ROOT = "/content/drive/MyDrive/Calorie_prediction_system/dataset/masks"
CHECKPOINT_PATH = "/content/drive/MyDrive/Calorie_prediction_system/sam_vit_b_01ec64.pth"

# Create mask directories if not exists
os.makedirs(MASK_ROOT, exist_ok=True)

# Load SAM model
sam = sam_model_registry["vit_b"](checkpoint=CHECKPOINT_PATH)
predictor = SamPredictor(sam)

# Process each image
missing = 0
processed = 0

for root, dirs, files in os.walk(IMAGE_ROOT):
    for file in tqdm(files):
        if not file.endswith(".jpg"):
            continue
        img_path = os.path.join(root, file)
        class_name = os.path.basename(os.path.dirname(img_path))
        mask_dir = os.path.join(MASK_ROOT, class_name)
        os.makedirs(mask_dir, exist_ok=True)
        mask_path = os.path.join(mask_dir, file)

        if os.path.exists(mask_path):
            continue  # skip already generated

        try:
            image = cv2.imread(img_path)
            if image is None:
                print(f"❌ Failed to read {img_path}")
                continue
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            predictor.set_image(image)

            height, width = image.shape[:2]
            input_point = np.array([[width//2, height//2]])
            input_label = np.array([1])

            masks, _, _ = predictor.predict(
                point_coords=input_point,
                point_labels=input_label,
                multimask_output=False
            )

            # Save binary mask (as grayscale)
            binary_mask = (masks[0] * 255).astype(np.uint8)
            cv2.imwrite(mask_path, binary_mask)
            processed += 1
        except Exception as e:
            print(f"Error on {img_path}: {e}")
            missing += 1

print(f"\n✅ Done generating masks.\nGenerated: {processed}\nFailed: {missing}")


0it [00:00, ?it/s]
 14%|█▎        | 136/1000 [1:37:21<10:18:33, 42.96s/it]


KeyboardInterrupt: 

In [5]:
!wget -P "/content/drive/MyDrive/Calorie_prediction_system/" https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth


--2025-07-06 13:15:57--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.173.166.51, 18.173.166.74, 18.173.166.31, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.173.166.51|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 375042383 (358M) [binary/octet-stream]
Saving to: ‘/content/drive/MyDrive/Calorie_prediction_system/sam_vit_b_01ec64.pth’

sam_vit_b_01ec64.pt 100%[===================>] 357.67M  48.5MB/s    in 8.9s    

2025-07-06 13:16:06 (40.1 MB/s) - ‘/content/drive/MyDrive/Calorie_prediction_system/sam_vit_b_01ec64.pth’ saved [375042383/375042383]



In [ ]:
import os
import shutil

def read_image_list(txt_file):
    with open(txt_file, "r") as f:
        lines = f.read().strip().splitlines()
    return lines

def resume_copy_images(image_list, src_dir, dst_dir):
    missing_count = 0
    for rel_path in image_list:
        class_name = rel_path.split('/')[0]
        file_name = os.path.basename(rel_path) + ".jpg"
        src = os.path.join(src_dir, class_name, file_name)
        dst_folder = os.path.join(dst_dir, class_name)
        dst = os.path.join(dst_folder, file_name)

        if not os.path.exists(dst):  # Only copy if not already present
            os.makedirs(dst_folder, exist_ok=True)
            try:
                shutil.copyfile(src, dst)
            except Exception as e:
                print(f"❌ Could not copy {src}: {e}")
                missing_count += 1
    print(f"✅ Copying complete. {missing_count} files failed to copy.")

# === Update Paths ===
IMAGE_ROOT = "/content/drive/MyDrive/Calorie_prediction_system/dataset/images"
TEST_DIR = "/content/drive/MyDrive/Calorie_prediction_system/dataset/test"
TEST_TXT = "/content/drive/MyDrive/Calorie_prediction_system/dataset/meta/test.txt"

# === Run ===
test_images = read_image_list(TEST_TXT)
resume_copy_images(test_images, IMAGE_ROOT, TEST_DIR)


✅ Copying complete. 0 files failed to copy.
